In [4]:
%load_ext autoreload
%autoreload 2
import bootstrap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from mxnet import ndarray as nd
import numpy as np
from dataclasses import dataclass

In [ ]:
# class BAM:
    
    
#     def __init__(n_inputs, n_outputs):
#         self._
        
    
    

In [ ]:
# step 1: replicate basic experiments from knoblauch chapter


In [ ]:
# define weight arrays
# later on make them sparse


In [10]:
n_u = 10  # input neurons
n_v = 10  # output neurons


u = nd.zeros([n_u])
v = nd.zeros([n_v])

W = nd.zeros([n_v, n_u])

In [11]:
W_potential = W.copy()

In [12]:
W_potential


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 10x10 @cpu(0)>

In [ ]:
@dataclass
class SynapseProperties:

    p_g = 0.1

    # probs named action-given-learning signal (e|s, c|s, d|s in the paper

    p_elim_1 = 0
    p_elim_0 = 0.01

    p_cons_1 = 0.97
    p_cons_0 = 0.03

    p_deco_0 = 0.8
    p_deco_1 = 0.2
    
    def __post_init__(self):

        # for now make sure these two add to 1
        assert np.isclose(1-self.p_cons_1, self.p_cons_0)
        assert np.isclose(1-self.p_deco_0, self.p_deco_0)

# @dataclass
# class AssemblyProperties:
    
#     k = 3
#     n = 10
    
#     # zip net rule parameter to set fraction of neuron's synapses that should be potentiated
#     p_1 = 0
    
#     def __post_init(self):
#         self.p_1 = self.k / self.n
    

In [ ]:
# balance / homeostatic constants

# ?? desired fraction of each neuron's synapses to be potentiated
#  ... how the f do we maintain this? does this drive the inhibition somehow?
# p_1 = 0.1 ???

# some definitions from the paper 

# P       - anatomical connectivity: fraction of neuron pairs in an assembly connected by 
#           at least one actual synapse
# P_pot   - fraction of neuron pairs connected by at least one potential synapse
# P_eff   - fraction of "required synapses" (given a learning signal S_ij) that 
#           have been realized

In [ ]:
# key stuff
# - cell assembly sizes must be limited to k for this to make sense, 
# thats a constraint on output patterns size if we want recurrent k-WTA inhibition to be 
# applied


In [ ]:
def fire(W_synapse, u_input, k_wta):
    """Returns binary activation after k-wta is applied on the postsynaptic neurons"""
    
    v = W_synapse * u_input
    
    # zero out activations that are not in the top-k    
    v = v.topk(k=k_wta, ret_type='mask')
    return v
        

In [31]:
def R_homosynaptic(u, v, p):
    """Returns a matrix of pairs of values from u and v 
    using the homosynaptic rule with probability (mean activation) p"""
    
    #return -p * 
    
    # TODO:
    # may be able to use khatri_rao and some fanciness to make this 
    # vectorized on gpu later
    
    W = nd.zeros([u.shape[0], v.shape[0]])
    for i, u_i in enumerate(u):
        for j, v_j in enumerate(v):
            W[i,j] = -p * u_i + (1*v_j)*u_i
    
    return W
    #return -p * u + (1 * v)*u
    

In [51]:
def allclose(a, b):    
    return nd.sum(nd.abs(a-b)) < 1e-6


In [52]:
assert allclose(R_homosynaptic(nd.array([0, 1]), nd.array([0, 1]), p=.1), nd.array([[0,0],[-0.1, 0.9]]))

In [54]:
R_homosynaptic(nd.array([1]), nd.array([1]), p=.1)


[[0.9]]
<NDArray 1x1 @cpu(0)>

In [ ]:
# TEST

class SimpleAssembly:
    
    properties: AssemblyProperties = AssemblyProperties()    
    
    def __init__(self, n_input, n_output, 
                 assembly_size, 
                 synapse_rule,
                 potentiated_synapse_fraction,
                
                ):
        self.n_u = n_input
        self.n_v = n_output
        
        self.R = synapse_rule
        self.k = assembly_size
        self.p_1 = potentiated_synapse_fraction
        self.u = nd.zeros([n_u])
        self.v = nd.zeros([n_v])

        self.W = nd.zeros([n_v, n_u])
        self.R_totals = W.copy()

    def fire(self, input_pattern):
        
        # check that its k or smaller in size?        
        self.v = fire(W_synapse=self.W, u_input=input_pattern, k_wta=self.k)
        return self.v

    def update_weights(self):
        
        # use zip net rule for now
        # TODO: figure out how the willshaw rule might apply in later stages of learning/
        #  consolidation 
        
        
        self.R_totals += self.synapse_rule(self.u, self.v, p=self.k/ self.n_output)
        
        
        # how to get just p1 synapses per (output) neuron potentiated?
        # well each output neuron is a column in v
        # number of active synapses is the number of non-zero values in the weight matrix
        #  NEXT STEP: finish implementing this dynamic thresholding
        #  -- do i really need to compute theta_ij dynamically for every neuron? like a second weight matrix?
        #  -- or can i just take the topk at every timestep...hmmmmmm
        
        self.W = self.R_totals - self.
    